In [ ]:
#load data
! pip install -q kaggle

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('key')
os.environ["KAGGLE_USERNAME"] = userdata.get('username')

In [ ]:
!kaggle competitions download -c liver-ultrasound-detection

In [ ]:
!unzip /content/liver-ultrasound-detection.zip

In [ ]:
# torch loaded...!!!
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# torchvision loaded...!!!
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchsummary import summary

# matplotlib loaded...!!!
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from tqdm import tqdm


In [ ]:
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 32
NOISE_FACTOR = 0.5 # constant NOISE_FACTOR which defines the amount of noise that we will add to our images.

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
from PIL import Image, ImageOps
from torch.utils.data import Dataset
from torchvision import datasets


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        #self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(glob.glob(self.img_dir+"/*"))

    def __getitem__(self, idx):
        img_path = glob.glob(self.img_dir+"/*")[idx]
        image = Image.open(img_path)
        image = ImageOps.grayscale(image)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, ), std=(0.5, ))
])

In [ ]:
train_data = CustomImageDataset('/content/train/train/images', transform)
trainLoader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
val_data = CustomImageDataset('/content/val/val/images', transform)
valLoader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
for batch in trainLoader:
    #batch1 = batch
    print(batch.shape)
    for img in batch:
        print(img.shape)
    break

In [ ]:
import glob
train_img_path = glob.glob("/content/train/train/images/*")
type(train_img_path)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def make_dir():
    image_dir = 'Saved_Images'
    if not os.path.exists(image_dir):
        os.mkdir(image_dir)

def save_decoded_image(img, name):
    #img = img.view(img.size(0), 1, 28, 28)
    save_image(img, 'Saved_Images/' + name + '.png')

In [ ]:
class AutorEncoder(nn.Module):
    def __init__(self):
        super(AutorEncoder, self).__init__()

        # Encoder layers
        self.enc1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.enc2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.enc3 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.enc4 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2,2)

        # Decoder layers
        self.dec1 = nn.ConvTranspose2d(8, 8, kernel_size=2, stride=2)
        self.dec2 = nn.ConvTranspose2d(8, 16, kernel_size=2, stride=2)
        self.dec3 = nn.ConvTranspose2d(16, 32, kernel_size=2, stride=2)
        self.dec4 = nn.ConvTranspose2d(32, 64, kernel_size=2, stride=2)
        self.out = nn.Conv2d(64, 1, kernel_size=3, padding=1)


    def forward(self, x):
        # Encoder
        x = self.pool(F.relu(self.enc1(x)))
        x = self.pool(F.relu(self.enc2(x)))
        x = self.pool(F.relu(self.enc3(x)))
        x = self.pool(F.relu(self.enc4(x))) # latent space representation

        # Decoder
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = torch.sigmoid(self.out(x))

        return x

model = AutorEncoder().to(device)

In [ ]:
summary(model, (1, 224, 224))

In [ ]:
# the loss function
loss_fn = nn.MSELoss()
# the optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-2)

In [ ]:
def train(net, trainLoader, NUM_EPOCHS):
    train_loss = []
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        with tqdm(trainLoader, unit="batch") as tepoch:
            for data in tepoch:
                tepoch.set_description(f"Epoch {epoch+1}")
                img = data # we do not need the image labels
                # add noise to the image data
                #img_noisy = img + NOISE_FACTOR * torch.randn(img.shape)
                # clip to make the values fall between 0 and 1
                #img_noisy = np.clip(img_noisy, 0., 1.)
                #img_noisy = img_noisy.to(device)
                img = np.clip(img, 0., 1.)
                img = img.to(device)
                optimizer.zero_grad()
                outputs = net(img)
                loss = loss_fn(outputs, img)
                # backpropagation
                loss.backward()
                # update the parameters
                optimizer.step()
                running_loss += loss.item()
                tepoch.set_postfix(loss=loss)

            loss = running_loss / len(trainLoader)
            train_loss.append(loss)
            # print(f"Epoch {epoch} loss: {loss}")
            save_decoded_image(img.cpu().data, name='noisy{}'.format(epoch))
            save_decoded_image(outputs.cpu().data, name='denoised{}'.format(epoch))

    return train_loss


def test_image_reconstruction(net, testloader):
     for batch in testloader:
        img = batch
        # img_noisy = img + NOISE_FACTOR * torch.randn(img.shape)
        img = np.clip(img, 0., 1.)
        img = img.to(device)
        outputs = net(img)
        outputs = outputs.view(outputs.size(0), 1, 224, 224).cpu().data
        save_image(img, 'noisy_test_input.png')
        save_image(outputs, 'denoised_test_reconstruction.png')
        break

In [ ]:
make_dir()
train_loss = train(model, trainLoader, 5)

In [ ]:
train_loss

In [ ]:
plt.figure()
plt.plot(train_loss)
plt.title('Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('./Saved_Images/conv_ae_fahsionmnist_loss.png')
test_image_reconstruction(model, valLoader)

In [ ]:
import matplotlib.image as mpimg
plt.figure(figsize=(20,20))
img = mpimg.imread("/content/Saved_Images/denoised0.png")
img1 = mpimg.imread("/content/Saved_Images/noisy0.png")
plt.subplot(1,2,1)
plt.imshow(img)
plt.title('Denoising Image')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(img1)
plt.title('Noisy Image')
plt.axis('off')
plt.show()